In [2]:
import numpy as np
from scipy import optimize
from abc import *

# Classe abstraite des resolveur d'equation 

In [3]:
class Resolver(ABC):
    """
    Lors de l'appelle d'un resolver une recine est retourner avec soit la presicion soit le nombre d'iteration
    les hypothese de convergence de chaque methode seront verifier par des assertions a 
    """
    @abstractclassmethod
    def __call__(self, precision=None, iteration=None, has_converged=None) -> tuple :
        pass
    
    

In [4]:
def methode_comparator(f):
        """
        Cette methode est un decorateur qui va tester et comparer notre Resolver a des fonctions de scipy
        """
        pass

In [5]:
from typing import Optional, Any
from dataclasses import dataclass

@dataclass
class Racine():
    racine : float
    precision : Optional[float]
    iteration : Optional[int]

In [6]:
class MaxtIterationExceded(BaseException):
    pass

class RacineOutOfIntervale(BaseException):
    pass

# La dichotomie

In [7]:


class Dichtomie(Resolver):

    def __new__(cls, f, a, b) :
        if f(a)*f(b) >0:
            raise RacineOutOfIntervale()
        obj = object.__new__(cls)
        obj.f = f
        obj.a, obj.b = min(a, b), max(a, b)
        return obj
            
        

    def __call__(self, precision=None, iteration=None, maxiter=10**9) -> Racine:
        assert precision != None or iteration != None, "precision et nombre d'iteration tous null" 
        
        f, a, b = self.f, self.a, self.b
        if precision:
            i = 0
            while np.abs(b-a) > precision:
                c = (a+b)/2
                if f(a)*f(c) < 0:
                    b = c
                else:
                    a = c
                i +=1
                if i == maxiter:
                    raise MaxtIterationExceded()
            return Racine(
                racine=(a+b)/2,
                precision=precision,
                iteration=i
            )
        
        if iteration:
            for i in range(iteration):
                c = (a+b)/2
                if f(a)*f(c) < 0:
                    b = c
                else:
                    a = c

            
            precision = (b-a)
            
            return Racine(
                racine=(a+b)/2,
                precision=precision,
                iteration=iteration
            )
        

In [8]:
f = lambda x : np.exp(-x/10)-x

dichtomie = Dichtomie(f, 0, 1) 
entete = "{:<20}|  {:<20}|  {:<20}|  {:<20}".format("np.exp(-x/10)-x", "racine calcule", "nombre d'iteration","precision" )
print(entete)

for precision in [10**-i for i in range(4, 10)]:
    racine = dichtomie(precision=precision)
    ligne= "{:<20}|  {:<20}|  {:<20}|  {:<20}".format("", racine.racine, racine.iteration, racine.precision)
    print(ligne)

np.exp(-x/10)-x     |  racine calcule      |  nombre d'iteration  |  precision           
                    |  0.912750244140625   |  14                  |  0.0001              
                    |  0.9127616882324219  |  17                  |  1e-05               
                    |  0.9127650260925293  |  20                  |  1e-06               
                    |  0.9127652943134308  |  24                  |  1e-07               
                    |  0.9127652682363987  |  27                  |  1e-08               
                    |  0.9127652714960277  |  30                  |  1e-09               


In [9]:

for iteration in [2**i for i in range(4, 10)]:
    racine = dichtomie(iteration=iteration)
    ligne= "{:<20}|  {:<20}|  {:<20}|  {:<20}".format("", racine.racine, racine.iteration, racine.precision)
    print(ligne)

                    |  0.9127578735351562  |  16                  |  1.52587890625e-05   
                    |  0.912765271612443   |  32                  |  2.3283064365386963e-10
                    |  0.9127652716086228  |  64                  |  0.0                 
                    |  0.9127652716086228  |  128                 |  0.0                 
                    |  0.9127652716086228  |  256                 |  0.0                 
                    |  0.9127652716086228  |  512                 |  0.0                 


# Fausse position

In [10]:
class LagrangeResolver(Resolver):

    def __new__(cls, f, a, b) :
        if f(a)*f(b) >0:
            raise RacineOutOfIntervale()
        obj = object.__new__(cls)
        obj.f = f
        obj.a, obj.b = min(a, b), max(a, b)
        return obj
            
        

    def __call__(self, precision=None, iteration=None, maxiter=10**9) -> Racine:
        assert precision != None or iteration != None, "precision et nombre d'iteration tous null" 

        f, a, b = self.f, self.a, self.b
        if precision:
            i = 0
            while np.abs(b-a) > precision:
                c = a -  f(a)*(a - b)/(f(a) - f(b))
                if f(a)*f(c) < 0:
                    b = c
                else:
                    a = c
                i +=1
                if i == maxiter:
                    raise MaxtIterationExceded()
            return Racine(
                racine=c,
                precision=precision,
                iteration=i
            )
        
        if iteration:
            for i in range(iteration):
                c = a -  f(a)*(a - b)/(f(a) - f(b))
                if f(a)*f(c) < 0:
                    b = c
                else:
                    a = c
            precision = np.min([np.abs(c-a), np.abs(c-b)])
            return Racine(
                racine=c,
                precision=precision,
                iteration=iteration
            )



In [11]:


lagrange = LagrangeResolver(f, 0, 1) 
entete = "{:<20}|  {:<20}|  {:<20}|  {:<20}".format("np.exp(-x/10)-x", "racine calcule", "nombre d'iteration","precision" )
print(entete)

for precision in [10**-i for i in range(5, 10)]:
    racine = lagrange(precision=precision)
    ligne= "{:<20}|  {:<20}|  {:<20}|  {:<20}".format("", racine.racine, racine.iteration, racine.precision)
    print(ligne)

np.exp(-x/10)-x     |  racine calcule      |  nombre d'iteration  |  precision           
                    |  0.9127652716086226  |  7                   |  1e-05               
                    |  0.9127652716086226  |  7                   |  1e-06               
                    |  0.9127652716086226  |  7                   |  1e-07               
                    |  0.9127652716086226  |  7                   |  1e-08               
                    |  0.9127652716086226  |  7                   |  1e-09               


In [12]:
print(entete)
for iteration in [2**i for i in range(1, 7)]:
    racine = dichtomie(iteration=iteration)
    ligne= "{:<20}|  {:<20}|  {:<20}|  {:<20}".format("", racine.racine, racine.iteration, racine.precision)
    print(ligne)

np.exp(-x/10)-x     |  racine calcule      |  nombre d'iteration  |  precision           
                    |  0.875               |  2                   |  0.25                
                    |  0.90625             |  4                   |  0.0625              
                    |  0.912109375         |  8                   |  0.00390625          
                    |  0.9127578735351562  |  16                  |  1.52587890625e-05   
                    |  0.912765271612443   |  32                  |  2.3283064365386963e-10
                    |  0.9127652716086228  |  64                  |  0.0                 


# Fibonacci

In [13]:
memory = np.array([1, 1])

def fibonacci(n):
    global memory
    if len(memory)>n:
        return memory[n]
    v = fibonacci(n-1)+fibonacci(n-2)
    memory = np.append(memory, v)
    return v

In [14]:
print(memory)

[1 1]


In [15]:
fibonacci(3)

3

In [16]:
fibonacci(8)

34

In [17]:
print(memory)


[ 1  1  2  3  5  8 13 21 34]


# Methode de la Secante

In [18]:
class SecanteResolver(Resolver):

    def __new__(cls, f, a, b) :
        if f(a)*f(b) >0:
            raise RacineOutOfIntervale()
        obj = object.__new__(cls)
        obj.f = f
        obj.a, obj.b = min(a, b), max(a, b)
        return obj
            
        

    def __call__(self, precision=None, iteration=None, maxiter=10**6) -> Racine:
        assert precision != None or iteration != None, "precision et nombre d'iteration tous null" 

        f, xm, xi = self.f, self.a, self.b
        #xm x i-1
        #xi x i 
        #xp x i+1
        if precision:
            i = 0
            while np.abs(xi-xm) > precision:
                xp= xm -  f(xm)*(xm - xi)/(f(xm) - f(xi))
                xm = xi
                xi = xp

                i +=1
                if i == maxiter:
                    raise MaxtIterationExceded()
            return Racine(
                racine=xi,
                precision=precision,
                iteration=i
            )
        
        if iteration:
            for i in range(iteration):
                if f(xm) - f(xi) == 0:
                    break
                xp= xm -  f(xm)*(xm - xi)/(f(xm) - f(xi))
                
                xm = xi
                xi = xp
                
            precision = np.abs(xi-xm)
            return Racine(
                racine=xp,
                precision=precision,
                iteration=iteration
            )


In [19]:


secante = SecanteResolver(f, 0, 1) 
entete = "{:<20}|  {:<20}|  {:<20}|  {:<20}".format("np.exp(-x/10)-x", "racine calcule", "nombre d'iteration","precision" )

print(entete)
for precision in [10**-i for i in range(5, 10)]:
    racine = secante(precision=precision)
    ligne= "{:<20}|  {:<20}|  {:<20}|  {:<20}".format("", racine.racine, racine.iteration, racine.precision)
    print(ligne)

np.exp(-x/10)-x     |  racine calcule      |  nombre d'iteration  |  precision           
                    |  0.9127652716087997  |  3                   |  1e-05               
                    |  0.9127652716087997  |  3                   |  1e-06               
                    |  0.9127652716086226  |  4                   |  1e-07               
                    |  0.9127652716086226  |  4                   |  1e-08               
                    |  0.9127652716086226  |  4                   |  1e-09               


In [20]:
print(entete)
for iteration in [2**i for i in range(1, 5)]:
    racine = secante(iteration=iteration)
    ligne= "{:<20}|  {:<20}|  {:<20}|  {:<20}".format("", racine.racine, racine.iteration, racine.precision)
    print(ligne)

np.exp(-x/10)-x     |  racine calcule      |  nombre d'iteration  |  precision           
                    |  0.9127651474614689  |  2                   |  0.0003412866595928765
                    |  0.9127652716086226  |  4                   |  1.7708057242771247e-13
                    |  0.9127652716086226  |  8                   |  0.0                 
                    |  0.9127652716086226  |  16                  |  0.0                 


## comparaison avec scipy.optimize.root_scaler(methode="secant")

In [21]:
from scipy.optimize import root_scalar

In [22]:
root = root_scalar(f,x0=0, x1=1, method="secant")
racine = secante(precision=10**(-10))


entete = "{:<20}|  {:<20}|  {:<20}|  {:<20}|  {:<20}".format("np.exp(-x/10)-x", "racine calcule","nombre d'iteration", "racine scipy", "nombre d'iteration scipy" )

print(entete)

ligne= "{:<20}|  {:<20}|  {:<20}|  {:<20}|  {:<20}".format("", racine.racine, racine.iteration, root.root, root.iterations)
print(ligne)

np.exp(-x/10)-x     |  racine calcule      |  nombre d'iteration  |  racine scipy        |  nombre d'iteration scipy
                    |  0.9127652716086226  |  4                   |  0.9127652716086226  |  4                   


# Methode de Newton

In [58]:
class NewtonResolver(Resolver):
    h = np.finfo(float).eps*4
    
    def __new__(cls, f, x0, a, b, ) :
        if f(a)*f(b) >0:
            raise RacineOutOfIntervale()
        a, b = min(a, b), max(a, b)
        assert a <= x0 <= b
        #assert NewtonResolver.derive_1(f, x0) *NewtonResolver.derive_2(f, x0) > 0
        # verification de la concavité
        obj = object.__new__(cls)
        obj.f = f
        obj.a, obj.b = min(a, b), max(a, b)
        obj.x0 = x0
        return obj

    @classmethod
    def derive_1(cls, f, x0):
        h = NewtonResolver.h
        return (f(x0+h) - f(x0-h))/(2*h)

    @classmethod
    def derive_2(cls, f, x0):
        h = NewtonResolver.h
        return (f(x0+h) + f(x0-h) - 2*f(x0))/(h**2)
    
            
        

    def __call__(self,fprime=None, precision=None, iteration=None, maxiter=10**6) -> Racine:
        assert precision != None or iteration != None, "precision et nombre d'iteration tous null" 

        f, a, b, xi = self.f, self.a, self.b, self.x0
        i = 1
        
        if not fprime:
            fprime = NewtonResolver.derive_1

        xp = xi - f(xi)/fprime(f, xi)
        if precision:
            while np.abs(xi - xp) > precision :
                xi = xp
                if fprime(f, xi) < NewtonResolver.h:
                    break

                xp =xi - f(xi)/fprime(f, xi)
                i += 1
                if i == maxiter:
                    raise MaxtIterationExceded("Nombre d'iteration maximal atteinte, racine obtenue ", xp)
            return Racine(
                racine=xp,
                precision=precision,
                iteration=i
            )
        
        if iteration:
            for i in range(iteration):
                xi = xp
                xp =xi - f(xi)/fprime(f, xi)
                
            precision = np.abs(xi-xp)
            return Racine(
                racine=xp,
                precision=precision,
                iteration=iteration
            )


## Resoltion de Newton avec frime de approximé

In [45]:
f =lambda x: (1/3)*x**3 - 2*x**2 + 3*x - (5/2)

newton = NewtonResolver(f,4.1, 4, 5) 
entete = "{:<35}|  {:<20}|  {:<20}|  {:<20}".format("(1/3)*x**3 - 2*x**2 + 3*x - (5/2)", "racine calcule", "nombre d'iteration","precision" )

print(entete)
for precision in [10**-i for i in range(5, 10)]:
    racine = newton(precision=precision)
    ligne= "{:<35}|  {:<20}|  {:<20}|  {:<20}".format("", racine.racine, racine.iteration, racine.precision)
    print(ligne)

(1/3)*x**3 - 2*x**2 + 3*x - (5/2)  |  racine calcule      |  nombre d'iteration  |  precision           
                                   |  4.317935804416695   |  14                  |  1e-05               
                                   |  4.3179259285967735  |  15                  |  1e-06               
                                   |  4.3179259285967735  |  15                  |  1e-07               
                                   |  4.3179259285967735  |  15                  |  1e-08               
                                   |  4.3179259285967735  |  15                  |  1e-09               


## Resoltion de Newton avec frime analytique

In [46]:

f =lambda x: (1/3)*x**3 - 2*x**2 + 3*x - (5/2)
fprime = lambda f, x : x**2 - 4*x + 3
newton = NewtonResolver(f,4.1, 4, 5) 
entete = "{:<35}|  {:<20}|  {:<20}|  {:<20}".format("(1/3)*x**3 - 2*x**2 + 3*x - (5/2)", "racine calcule", "nombre d'iteration","precision" )

print(entete)
for precision in [10**-i for i in range(5, 10)]:
    racine = newton(fprime=fprime, precision=precision)
    ligne= "{:<35}|  {:<20}|  {:<20}|  {:<20}".format("", racine.racine, racine.iteration, racine.precision)
    print(ligne)

(1/3)*x**3 - 2*x**2 + 3*x - (5/2)  |  racine calcule      |  nombre d'iteration  |  precision           
                                   |  4.317931287501243   |  3                   |  1e-05               
                                   |  4.317931287501243   |  3                   |  1e-06               
                                   |  4.317931287501235   |  4                   |  1e-07               
                                   |  4.317931287501235   |  4                   |  1e-08               
                                   |  4.317931287501235   |  4                   |  1e-09               


## comparaison avec scipy.optimize.root_scaler(methode="secant")

# 

In [ ]:
from scipy.optimize import root_scalar

In [60]:
fprime_scpiy = lambda  x : x**2 - 4*x + 3
fprime = lambda f, x: fprime_scpiy(x)

racine = newton(fprime=fprime, precision=10**(-10))

entete = "{:<35}|  {:<20}|  {:<20}|  {:<20}|  {:<20}".format("(1/3)*x**3 - 2*x**2 + 3*x - (5/2)", "racine calcule","nombre d'iteration", "racine scipy", "nombre d'iteration scipy")

print(entete)

ligne= "{:<35}|  {:<20}|  {:<20}|  {:<20}|  {:<20}".format("", racine.racine, racine.iteration, root.root, root.iterations)
print(ligne)

(1/3)*x**3 - 2*x**2 + 3*x - (5/2)  |  racine calcule      |  nombre d'iteration  |  racine scipy        |  nombre d'iteration scipy
                                   |  4.317931287501235   |  4                   |  4.317931287501235   |  5                   


# Methode du point fixe

In [83]:
class FixedPoint(Resolver):
    
    def __new__(cls, f, g, a, b,  x0=None) :
        
        a, b = min(a, b), max(a, b)
        if not x0:
            x0 = a
        else:
            assert a <= x0 <= b

        
        
        interval = np.linspace(a, b, 100)
        interval_img = NewtonResolver.derive_1(g, interval)
        interval_img = np.abs(interval_img)
        if not all(interval_img < 1):
            raise RacineOutOfIntervale("Le resolution est impossible sur cette intervale : g n'est pas contracte")
        
        
        
       
        obj = object.__new__(cls)
        obj.f = f
        obj.g = g
        obj.a, obj.b = min(a, b), max(a, b)
        obj.x0 = x0
        return obj
    
            
        

    def __call__(self, fprime=None, precision=None, iteration=None, maxiter=10**6) -> Racine:
        assert precision != None or iteration != None, "precision et nombre d'iteration tous null" 

        f,g, a, b, xi = self.f, self.g, self.a, self.b, self.x0
        i = 1

        if not fprime:
            fprime = NewtonResolver.derive_1

        xp = g(xi)
        if precision:
            while np.abs(xi - xp) > precision :
                xi = xp
                xp = g(xi)
                i += 1
                if i == maxiter:
                    raise MaxtIterationExceded("Nombre d'iteration maximal atteinte, racine obtenue ", xp)
            return Racine(
                racine=xp,
                precision=precision,
                iteration=i
            )
        
        if iteration:
            for i in range(iteration):
                xi = xp
                xp = g(xi)
                
            precision = np.abs(xi-xp)
            return Racine(
                racine=xp,
                precision=precision,
                iteration=iteration
            )


In [76]:
try:
    f =lambda x: (1/3)*x**3 - 2*x**2 + 3*x - (5/2)
    g = lambda x : (-x**3)/9 + (2/3)*x**2 + 5/6
    fixed_point = FixedPoint(f, g,  4, 5, x0=4.1) 
    entete = "{:<35}|  {:<20}|  {:<20}|  {:<20}".format("(1/3)*x**3 - 2*x**2 + 3*x - (5/2)", "racine calcule", "nombre d'iteration","precision" )

    print(entete)
    for precision in [10**-i for i in range(5, 10)]:
        racine = fixed_point( precision=precision)
        ligne= "{:<35}|  {:<20}|  {:<20}|  {:<20}".format("", racine.racine, racine.iteration, racine.precision)
        print(ligne)
except RacineOutOfIntervale as e:
    print(e.args[0])


Le resolution est impossible sur cette intervale : g n'est pas contracte


In [77]:
try:
    f =lambda x: (1/3)*x**3 - 2*x**2 + 3*x - (5/2)
    g = lambda x : ((1/6)*x**3 + (2/3)*x -5/4)**.5
    fixed_point = FixedPoint(f, g,  4, 5, x0=4.1) 
    entete = "{:<35}|  {:<20}|  {:<20}|  {:<20}".format("(1/3)*x**3 - 2*x**2 + 3*x - (5/2)", "racine calcule", "nombre d'iteration","precision" )

    print(entete)
    for precision in [10**-i for i in range(5, 10)]:
        racine = fixed_point( precision=precision)
        ligne= "{:<35}|  {:<20}|  {:<20}|  {:<20}".format("", racine.racine, racine.iteration, racine.precision)
        print(ligne)
except RacineOutOfIntervale as e:
    print(e.args[0])



Le resolution est impossible sur cette intervale : g n'est pas contracte


In [85]:
try:
    f =lambda x: np.exp(-.1*x) - x
    g = lambda x : np.exp(-.1*x)
    fixed_point = FixedPoint(f, g,  1, 2) 
    entete = "{:<20}|  {:<20}|  {:<20}|  {:<20}".format("np.exp(-.1*x) - x", "racine calcule", "nombre d'iteration","precision" )

    print(entete)
    for precision in [10**-i for i in range(5, 10)]:
        racine = fixed_point( precision=precision)
        ligne= "{:<20}|  {:<20}|  {:<20}|  {:<20}".format("", racine.racine, racine.iteration, racine.precision)
        print(ligne)
except RacineOutOfIntervale as e:
    print(e.args[0])



np.exp(-.1*x) - x   |  racine calcule      |  nombre d'iteration  |  precision           
                    |  0.9127647211181605  |  5                   |  1e-05               
                    |  0.9127653218554816  |  6                   |  1e-06               
                    |  0.9127652670222639  |  7                   |  1e-07               
                    |  0.9127652720272496  |  8                   |  1e-08               
                    |  0.9127652715704119  |  9                   |  1e-09               
